# DOWNLOAD sucessful, LOADING Models Weights from Disk FAILED!

Script Useful for downloading model weights in the future. **(Correction: Easier to just git clone the repo) - doesn't work for 70B**

Running model from cache works. But script fails to load model weights from disk.
Possible reasons why loading from disk failed:
- safetensors?
- shared tensors
- unsafe serialization

use_safetensors=True, 

safe_serialization=False,

RuntimeError: The weights trying to be saved contained shared tensors

### DOWNLOAD LLM WEIGHTS

In [ ]:
import os
from huggingface_hub import hf_hub_download, login, whoami
import dotenv

dotenv.load_dotenv()
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_API_KEY")
login(token=HUGGINGFACE_TOKEN)

# Download the model from the Hub
model_id = "google/gemma-2b"
filenames = [
        "config.json",
        "gemma-2b.gguf",
        "generation_config.json",
        "model-00001-of-00002.safetensors",
        "model-00001-of-00002.safetensors",
        "model.safetensors.index.json",
        "special_tokens_map.json",
        "tokenizer.json",
        "tokenizer.model",
        "tokenizer_config.json"
]
for filename in filenames:
    downloaded_model_path = hf_hub_download(
        repo_id=model_id, filename=filename, revision="main", token=HUGGINGFACE_TOKEN
    )
    print(f"Downloaded {downloaded_model_path}")



### RUN LLM

In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", use_safetensors=True)


In [ ]:
# import torch

# # Saving model weights to disk
# tokenizer.save_pretrained("gemma_model_saved")

# # Save the model using torch.save
# model.save_pretrained("gemma_model_saved", use_safetensors=True, safe_serialization=False)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# tokenizer = AutoTokenizer.from_pretrained("gemma_model_saved")
# model = AutoModelForCausalLM.from_pretrained("gemma_model_saved")

# To load the model
path = "gemma_model_saved"
model = AutoModelForCausalLM.from_pretrained(path, use_safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(path)

/home/dpaul/prompt-linter/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

In [ ]:
# chat = [
#     { "role": "user", "content": "What is 2+2?" },
# ]

# prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# prompt

In [ ]:
prompt = "What is 2+2?"
input_text = f"""<start_of_turn>user
{prompt}<end_of_turn>
<start_of_turn>model"""
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=128)
print(tokenizer.decode(outputs[0]))